In this example, we will use CamelAI to simulate tools! In this case, we will best determine how many shots it takes to beat an enemy with a blue shield in Apex Legeneds using melee only. The character "Pathfinder" from Apex Legends will answer.

First let's install the dependencies for AgentOps/CamelAI

In [ ]:
%pip install camel-ai[all]==0.2.9
%pip install agentops==0.3.10

Setup our imports (As usual)

In [2]:
import agentops
import os
from getpass import getpass

And we set our API keys

In [3]:
# Prompt for the OpenAI API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your API key:  ········


In [4]:
# Prompt for the AgentOps API key securely
agentops_api_key = getpass('Enter your API key: ')
os.environ["AGENTOPS_API_KEY"] = agentops_api_key

Enter your API key:  ········


We import a few more things Camel needs

In [5]:
from typing import List

from colorama import Fore

from camel.agents.chat_agent import FunctionCallingRecord
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory
from camel.societies import RolePlaying
from camel.types import ModelPlatformType, ModelType
from camel.utils import print_text_animated

import agentops

Now we setup our assistant!

In [ ]:
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY")
agentops.init(tags=["CAMEL X AgentOps Multi-agent with Tools"])


from camel.toolkits import (
    SearchToolkit,
    MathToolkit,
)

And now we set a task prompt!

In [7]:
task_prompt = (
    "We are in the Apex Legends Games. Determine the amount of"
    "meele hits it will take to beat someone with a blue shield."
    "You should answer as if you are Pathfinder from the Apex Games."
)

And let's set our tools! You can look at the link below to see all available tools;

https://docs.camel-ai.org/key_modules/tools.html

In [11]:
tools = [
    *MathToolkit().get_tools(),
    *SearchToolkit().get_tools(),
]

Finally, let's setup our session!

In [12]:
search_session = RolePlaying(
    assistant_role_name="Searcher",
    user_role_name="Pathfinder",
    assistant_agent_kwargs=dict(
        model=ModelFactory.create(
            model_platform=ModelPlatformType.OPENAI,
            model_type=ModelType.GPT_4O_MINI,
        ),
        tools=tools,
    ),
    user_agent_kwargs=dict(
        model=ModelFactory.create(
            model_platform=ModelPlatformType.OPENAI,
            model_type=ModelType.GPT_4O_MINI,
        ),
    ),
    task_prompt=task_prompt,
    with_task_specify=False,
)

System Message and Task Prompt

In [15]:
print(
    Fore.GREEN
    + f"AI Assistant sys message:\n{search_session.assistant_sys_msg}\n"
)
print(Fore.BLUE + f"AI User sys message:\n{search_session.user_sys_msg}\n")

print(Fore.YELLOW + f"Original task prompt:\n{task_prompt}\n")
print(
    Fore.CYAN
    + "Specified task prompt:"
    + f"\n{search_session.specified_task_prompt}\n"
)
print(Fore.RED + f"Final task prompt:\n{search_session.task_prompt}\n")

AI Assistant sys message:
BaseMessage(role_name='Searcher', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={'task': 'We are in the Apex Legends Games. Determine the amount ofmeele hits it will take to beat someone with a blue shield.You should answer as if you are Pathfinder from the Apex Games.', 'assistant_role': 'Searcher', 'user_role': 'Pathfinder'}, content='===== RULES OF ASSISTANT =====\nNever forget you are a Searcher and I am a Pathfinder. Never flip roles! Never instruct me!\nWe share a common interest in collaborating to successfully complete a task.\nYou must help me to complete the task.\nHere is the task: We are in the Apex Legends Games. Determine the amount ofmeele hits it will take to beat someone with a blue shield.You should answer as if you are Pathfinder from the Apex Games.. Never forget our task!\nI must instruct you based on your expertise and my needs to complete the task.\n\nI must give you one instruction at a time.\nYou must write a specific solution

In [17]:
n = 0
input_msg = search_session.init_chat()
while n < 50:
    n += 1
    assistant_response, user_response = search_session.step(input_msg)

    if assistant_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI Assistant terminated. Reason: "
                f"{assistant_response.info['termination_reasons']}."
            )
        )
        break
    if user_response.terminated:
        print(
            Fore.GREEN
            + (
                "AI User terminated. "
                f"Reason: {user_response.info['termination_reasons']}."
            )
        )
        break

    # Print output from the user
    print_text_animated(
        Fore.BLUE + f"AI User:\n\n{user_response.msg.content}\n"
    )

    # Print output from the assistant, including any function
    # execution information
    print_text_animated(Fore.GREEN + "AI Assistant:")
    tool_calls: List[FunctionCallingRecord] = assistant_response.info[
        'tool_calls'
    ]
    for func_record in tool_calls:
        print_text_animated(f"{func_record}")
    print_text_animated(f"{assistant_response.msg.content}\n")

    if "CAMEL_TASK_DONE" in user_response.msg.content:
        break

    input_msg = assistant_response.msg

AI User:

Instruction: Determine the amount of melee damage required to break a blue shield.  
Input: None


AI Assistant:

Solution: To determine the amount of melee damage required to break a blue shield in Apex Legends, we first need to understand the mechanics of shields in the game.

In Apex Legends, shields are categorized by color, with blue shields providing 50 points of protection. Melee attacks deal a fixed amount of damage, which is 30 damage per hit.

To calculate how many melee hits it would take to break a blue shield, we can use the following formula:

1. **Total Shield Health**: 50 (for blue shield)
2. **Melee Damage per Hit**: 30

Now, we divide the total shield health by the melee damage per hit:

\[
\text{Number of Hits} = \frac{\text{Total Shield Health}}{\text{Melee Damage per Hit}} = \frac{50}{30} \approx 1.67
\]

Since you cannot hit a fraction of a time, you will need to round up to the nearest whole number. Therefore, it will take **2 melee hits** to break a bl

KeyboardInterrupt: 

In [18]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.001192
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=5052a276-92eb-46c2-b606-45630ad99b91
